In [23]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data/data12632/


first_round_testin  first_round_traini	submit_example.csv


In [24]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

result.csv


In [25]:
! pip install catboost

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/


In [26]:
! pip install mlxtend

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/


In [27]:
! pip install xgboost

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/


In [28]:
! pip install lightgbm

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/


In [29]:
# ! pip install tqdm

In [30]:
# ! pip install auto-sklearn

In [31]:
# !pip list


In [32]:
import catboost as cbt
import numpy as np
import pandas as pd
from mlxtend.classifier import StackingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
# from tqdm import tqdm
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import time
import copy
import gc
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [33]:
train_data = pd.read_csv('/home/aistudio/data/data12632/first_round_traini')
test_data = pd.read_csv('/home/aistudio/data/data12632/first_round_testin', encoding='utf-8')
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
train_data['Quality_label'] = train_data['Quality_label'].map(dit)
labels = pd.get_dummies(train_data['Quality_label']).values
submit = pd.read_csv('/home/aistudio/data/data12632/submit_example.csv')

In [34]:
# features = ['Parameter5', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10']
# train_data[features] = np.log(train_data[features] + 1e-5)
# test_data[features] = np.log(test_data[features] + 1e-5)
# scale_features = []
# for feature in features:
#     for feature1 in features:
#         train_data[str(feature)+"_"+str(feature1)] = train_data[feature]+train_data[feature1]
#         train_data[str(feature)+"mul"+str(feature1)] = train_data[feature].mul(train_data[feature1])
#         train_data[str(feature)+"%"+str(feature1)] = train_data[feature]%train_data[feature1]
#         train_data[str(feature)+"/"+str(feature1)] = train_data[feature]/train_data[feature1]
        
#         scale_features.append(str(feature)+"_"+str(feature1))
#         scale_features.append(str(feature)+"mul"+str(feature1))
#         scale_features.append(str(feature)+"%"+str(feature1))
#         scale_features.append(str(feature)+"/"+str(feature1))

# # 组合所有特征
# features = features + scale_features
# print(features)



In [35]:
# 取parameter5-10，取log
features = ['Parameter5', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10']
train_data[features] = np.log(train_data[features] + 1e-5)
test_data[features] = np.log(test_data[features] + 1e-5)
# 特征融合
# train_data['5_6'] = train_data['Parameter5'] + train_data['Parameter6']
# test_data['5_6'] = test_data['Parameter5'] + test_data['Parameter6']
train_data['5_7'] = train_data['Parameter5'] + train_data['Parameter7']
test_data['5_7'] = test_data['Parameter5'] + test_data['Parameter7']
train_data['7_8'] = train_data['Parameter7'] + train_data['Parameter8']
test_data['7_8'] = test_data['Parameter7'] + test_data['Parameter8']
train_data['7_9'] = train_data['Parameter7'] + train_data['Parameter9']
test_data['7_9'] = test_data['Parameter7'] + test_data['Parameter9']
train_data['7_10'] = train_data['Parameter7'] + train_data['Parameter10']
test_data['7_10'] = test_data['Parameter7'] + test_data['Parameter10']
train_data['8_9'] = train_data['Parameter8'] + train_data['Parameter9']
test_data['8_9'] = test_data['Parameter8'] + test_data['Parameter9']
train_data['8_10'] = train_data['Parameter8'] + train_data['Parameter10']
test_data['8_10'] = test_data['Parameter8'] + test_data['Parameter10']
train_data['9_10'] = train_data['Parameter9'] + train_data['Parameter10']
test_data['9_10'] = test_data['Parameter9'] + test_data['Parameter10']

train_data['5_10'] = train_data['Parameter5'] + train_data['Parameter10']
test_data['5_10'] = test_data['Parameter5'] + test_data['Parameter10']
# train_data['6_10'] = train_data['Parameter6'] + train_data['Parameter10']
# test_data['6_10'] = test_data['Parameter6'] + test_data['Parameter10']
train_data['7_8_9'] = train_data['Parameter7'] + train_data['Parameter8'] + train_data['Parameter9']
test_data['7_8_9'] = test_data['Parameter7'] + test_data['Parameter8'] + test_data['Parameter9']
train_data['7_9_10'] = train_data['Parameter7'] + train_data['Parameter9'] + train_data['Parameter10']
test_data['7_9_10'] = test_data['Parameter7'] + test_data['Parameter9'] + test_data['Parameter10']
train_data['7_8_10'] = train_data['Parameter7'] + train_data['Parameter8'] + train_data['Parameter10']
test_data['7_8_10'] = test_data['Parameter7'] + test_data['Parameter8'] + test_data['Parameter10']
train_data['8_9_10'] = train_data['Parameter8'] + train_data['Parameter9'] + train_data['Parameter10']
test_data['8_9_10'] = test_data['Parameter8'] + test_data['Parameter9'] + test_data['Parameter10']
train_data['7_8_9_10'] = train_data['Parameter7'] + train_data['Parameter8'] + train_data['Parameter9'] + train_data['Parameter10']
test_data['7_8_9_10'] = test_data['Parameter7'] + test_data['Parameter8'] + test_data['Parameter9'] + test_data['Parameter10']

# train_data['5-6'] = (train_data['Parameter5'] - train_data['Parameter6'])
# test_data['5-6'] = (test_data['Parameter5'] - test_data['Parameter6'])
train_data['7-8'] = (train_data['Parameter7'] - train_data['Parameter8'])
test_data['7-8'] = (test_data['Parameter7'] - test_data['Parameter8'])
train_data['7-9'] = (train_data['Parameter7'] - train_data['Parameter9'])
test_data['7-9'] = (test_data['Parameter7'] - test_data['Parameter9'])
train_data['7-10'] = (train_data['Parameter7'] - train_data['Parameter10'])
test_data['7-10'] = (test_data['Parameter7'] - test_data['Parameter10'])
train_data['8-9'] = (train_data['Parameter8'] - train_data['Parameter9'])
test_data['8-9'] = (test_data['Parameter8'] - test_data['Parameter9'])
train_data['8-10'] = (train_data['Parameter8'] - train_data['Parameter10'])
test_data['8-10'] = (test_data['Parameter8'] - test_data['Parameter10'])
train_data['9-10'] = (train_data['Parameter9'] - train_data['Parameter10'])
test_data['9-10'] = (test_data['Parameter9'] - test_data['Parameter10'])

# train_data['5_6_mul'] = train_data['Parameter5'].mul(train_data['Parameter6'])
# test_data['5_6_mul'] = test_data['Parameter5'].mul(test_data['Parameter6'])
train_data['7_8_mul'] = train_data['Parameter7'].mul(train_data['Parameter8'])
test_data['7_8_mul'] = test_data['Parameter7'].mul(test_data['Parameter8'])
train_data['7_9_mul'] = train_data['Parameter7'].mul(train_data['Parameter9'])
test_data['7_9_mul'] = test_data['Parameter7'].mul(test_data['Parameter9'])
train_data['7_10_mul'] = train_data['Parameter7'].mul(train_data['Parameter10'])
test_data['7_10_mul'] = test_data['Parameter7'].mul(test_data['Parameter10'])
train_data['8_9_mul'] = train_data['Parameter8'].mul(train_data['Parameter9'])
test_data['8_9_mul'] = test_data['Parameter8'].mul(test_data['Parameter9'])
train_data['8_10_mul'] = train_data['Parameter8'].mul(train_data['Parameter10'])
test_data['8_10_mul'] = test_data['Parameter8'].mul(test_data['Parameter10'])
train_data['9_10_mul'] = train_data['Parameter9'].mul(train_data['Parameter10'])
test_data['9_10_mul'] = test_data['Parameter9'].mul(test_data['Parameter10'])

train_data['78_mul_79'] = (train_data['7_8'].mul(train_data['7_9']))
test_data['78_mul_79'] = (test_data['7_8'].mul(test_data['7_9']))
train_data['78_mul_710'] = (train_data['7_8'].mul(train_data['7_10']))
test_data['78_mul_710'] = (test_data['7_8'].mul(test_data['7_10']))
train_data['78_mul_89'] = (train_data['7_8'].mul(train_data['8_9']))
test_data['78_mul_89'] = (test_data['7_8'].mul(test_data['8_9']))
train_data['78_mul_810'] = (train_data['7_8'].mul(train_data['8_10']))
test_data['78_mul_810'] = (test_data['7_8'].mul(test_data['8_10']))
train_data['78_mul_910'] = (train_data['7_8'].mul(train_data['9_10']))
test_data['78_mul_910'] = (test_data['7_8'].mul(test_data['9_10']))
train_data['79_mul_710'] = (train_data['7_9'].mul(train_data['7_10']))
test_data['79_mul_710'] = (test_data['7_9'].mul(test_data['7_10']))
train_data['79_mul_89'] = (train_data['7_9'].mul(train_data['8_9']))
test_data['79_mul_89'] = (test_data['7_9'].mul(test_data['8_9']))
train_data['79_mul_810'] = (train_data['7_9'].mul(train_data['8_10']))
test_data['79_mul_810'] = (test_data['7_9'].mul(test_data['8_10']))
train_data['79_mul_910'] = (train_data['7_9'].mul(train_data['9_10']))
test_data['79_mul_910'] = (test_data['7_9'].mul(test_data['9_10']))
train_data['710_mul_89'] = (train_data['7_10'].mul(train_data['8_9']))
test_data['710_mul_89'] = (test_data['7_10'].mul(test_data['8_9']))
train_data['710_mul_810'] = (train_data['7_10'].mul(train_data['8_10']))
test_data['710_mul_810'] = (test_data['7_10'].mul(test_data['8_10']))
train_data['710_mul_910'] = (train_data['7_10'].mul(train_data['9_10']))
test_data['710_mul_910'] = (test_data['7_10'].mul(test_data['9_10']))
train_data['89_mul_810'] = (train_data['8_9'].mul(train_data['8_10']))
test_data['89_mul_810'] = (test_data['8_9'].mul(test_data['8_10']))
train_data['89_mul_910'] = (train_data['8_9'].mul(train_data['9_10']))
test_data['89_mul_910'] = (test_data['8_9'].mul(test_data['9_10']))
train_data['810_mul_910'] = (train_data['8_10'].mul(train_data['9_10']))
test_data['810_mul_910'] = (test_data['8_10'].mul(test_data['9_10']))

train_data['5_8'] = train_data['Parameter5'] + train_data['Parameter8']
test_data['5_8'] = test_data['Parameter5'] + test_data['Parameter8']
train_data['7+8*8'] = train_data['Parameter7'] + train_data['Parameter8'] * train_data['Parameter8']
test_data['7+8*8'] = test_data['Parameter7'] + test_data['Parameter8'] * test_data['Parameter8']
train_data['7*7+8'] = train_data['Parameter7'] * train_data['Parameter7'] + train_data['Parameter8']
test_data['7*7+8'] = test_data['Parameter7'] * test_data['Parameter7'] + test_data['Parameter8']

train_data['9/7'] = train_data['Parameter9'] / train_data['Parameter7']
test_data['9/7'] = test_data['Parameter9'] / test_data['Parameter7']
train_data['9/8'] = train_data['Parameter9'] / train_data['Parameter8']
test_data['9/8'] = test_data['Parameter9'] / test_data['Parameter8']
train_data['1/7'] = 1. / train_data['Parameter7']
test_data['1/7'] = 1. / test_data['Parameter7']
train_data['9%8'] = train_data['Parameter9'] % train_data['Parameter8']
test_data['9%8'] = test_data['Parameter9'] % test_data['Parameter8']
train_data['7%9'] = train_data['Parameter7'] % train_data['Parameter9']
test_data['7%9'] = test_data['Parameter7'] % test_data['Parameter9']
train_data['1/9'] = 1. / train_data['Parameter9']
test_data['1/9'] = 1. / test_data['Parameter9']
train_data['7/8'] = train_data['Parameter7'] / train_data['Parameter8']
test_data['7/8'] = test_data['Parameter7'] / test_data['Parameter8']
train_data['5%7'] = train_data['Parameter5'] % train_data['Parameter7']
test_data['5%7'] = test_data['Parameter5'] % test_data['Parameter7']
train_data['8%9'] = train_data['Parameter8'] % train_data['Parameter9']
test_data['8%9'] = test_data['Parameter8'] % test_data['Parameter9']
train_data['7/9'] = train_data['Parameter7'] / train_data['Parameter9']
test_data['7/9'] = test_data['Parameter7'] / test_data['Parameter9']
train_data['8/7'] = train_data['Parameter8'] / train_data['Parameter7']
test_data['8/7'] = test_data['Parameter8'] / test_data['Parameter7']
train_data['1/8'] = 1. / train_data['Parameter8']
test_data['1/8'] = 1. / test_data['Parameter8']
train_data['8%7'] = train_data['Parameter8'] % train_data['Parameter7']
test_data['8%7'] = test_data['Parameter8'] % test_data['Parameter7']
train_data['8+9/8%9'] = (train_data['Parameter8'] + train_data['Parameter9']) / train_data['Parameter8'] % train_data['Parameter9']
test_data['8+9/8%9'] = (test_data['Parameter8'] + test_data['Parameter9']) / test_data['Parameter8'] % test_data['Parameter9']
train_data['7%8'] = train_data['Parameter7'] % train_data['Parameter8']
test_data['7%8'] = test_data['Parameter7'] % test_data['Parameter8']
train_data['8/9'] = train_data['Parameter8'] / train_data['Parameter9']
test_data['8/9'] = test_data['Parameter8'] / test_data['Parameter9']

train_data['7+8%7+9'] = train_data['Parameter7'] + train_data['Parameter8'] % train_data['Parameter7'] + train_data['Parameter9']
test_data['7+8%7+9'] = test_data['Parameter7'] + test_data['Parameter8'] % test_data['Parameter7'] + test_data['Parameter9']
train_data['1/8/9'] = 1./train_data['Parameter8']/train_data['Parameter9']
test_data['1/8/9'] = 1./test_data['Parameter8']/test_data['Parameter9']
train_data['9%7/9'] = train_data['Parameter9']%train_data['Parameter7']/train_data['Parameter9']
test_data['9%7/9'] = test_data['Parameter9']%test_data['Parameter7']/test_data['Parameter9']
train_data['8+9/8*9'] = train_data['Parameter8']+train_data['Parameter9']/train_data['Parameter8']*train_data['Parameter9']
test_data['8+9/8*9'] = test_data['Parameter8']+test_data['Parameter9']/test_data['Parameter8']*test_data['Parameter9']
train_data['9%7/7*8'] = train_data['Parameter9']%train_data['Parameter7']/train_data['Parameter7']*train_data['Parameter8']
test_data['9%7/7*8'] = test_data['Parameter9']%test_data['Parameter7']/test_data['Parameter7']*test_data['Parameter8']
train_data['7%8/7'] = train_data['Parameter7']%train_data['Parameter8']/train_data['Parameter7']
test_data['7%8/7'] = test_data['Parameter7']%test_data['Parameter8']/test_data['Parameter7']
train_data['9%8*9'] = train_data['Parameter9']%train_data['Parameter8']*train_data['Parameter9']
test_data['9%8*9'] = test_data['Parameter9']%test_data['Parameter8']*test_data['Parameter9']
train_data['8*9%7'] = train_data['Parameter8']*train_data['Parameter9']%train_data['Parameter7']
test_data['8*9%7'] = test_data['Parameter8']*test_data['Parameter9']%test_data['Parameter7']

train_data['8%9*8'] = train_data['Parameter8']%train_data['Parameter9']*train_data['Parameter8']
test_data['8%9*8'] = test_data['Parameter8']%test_data['Parameter9']*test_data['Parameter8']
train_data['7*8*9'] = train_data['Parameter8']*train_data['Parameter7']*train_data['Parameter9']
test_data['7*8*9'] = test_data['Parameter8']*test_data['Parameter7']*test_data['Parameter9']
train_data['8+9/7*8']=train_data['Parameter8']+train_data['Parameter9']/train_data['Parameter7']*train_data['Parameter9']
test_data['8+9/7*8']=test_data['Parameter8']+test_data['Parameter9']/test_data['Parameter7']*test_data['Parameter9']
train_data['8%7/(9%8)'] = train_data['Parameter8']%train_data['Parameter7']/(train_data['Parameter9']%train_data['Parameter8'])
test_data['8%7/(9%8)'] = test_data['Parameter8']%test_data['Parameter7']/(test_data['Parameter9']%test_data['Parameter8'])
train_data['7%8/(8%9)'] = train_data['Parameter7']%train_data['Parameter8']/(train_data['Parameter8']%train_data['Parameter9'])
test_data['7%8/(8%9)'] = test_data['Parameter7']%test_data['Parameter8']/(test_data['Parameter8']%test_data['Parameter9'])
train_data['8+9/(9%8)']=train_data['Parameter8']+train_data['Parameter9']/(train_data['Parameter9']%train_data['Parameter8'])
test_data['8+9/(9%8)']=test_data['Parameter8']+test_data['Parameter9']/(test_data['Parameter9']%test_data['Parameter8'])
train_data['8*(9%7)+8'] = train_data['Parameter8']*(train_data['Parameter9']%train_data['Parameter7']+train_data['Parameter8'])
test_data['8*(9%7)+8'] = test_data['Parameter8']*(test_data['Parameter9']%test_data['Parameter7']+test_data['Parameter8'])
train_data['7+8*9'] = train_data['Parameter7']+train_data['Parameter8']*train_data['Parameter9']
test_data['7+8*9'] = test_data['Parameter7']+test_data['Parameter8']*test_data['Parameter9']
train_data['7%8/(9%8)'] = train_data['Parameter7']%train_data['Parameter8']/(train_data['Parameter9']%train_data['Parameter8'])
test_data['7%8/(9%8)'] = test_data['Parameter7']%test_data['Parameter8']/(test_data['Parameter9']%test_data['Parameter8'])
train_data['9%8/(7%8)'] = train_data['Parameter9']%train_data['Parameter8']/(train_data['Parameter7']%train_data['Parameter8'])
test_data['9%8/(7%8)'] = test_data['Parameter9']%test_data['Parameter8']/(test_data['Parameter7']%test_data['Parameter8'])
train_data['5%8'] = train_data['Parameter5']%train_data['Parameter8']
test_data['5%8'] = test_data['Parameter5']%test_data['Parameter8']
# train_data['6%8'] = train_data['Parameter6']%train_data['Parameter8']
# test_data['6%8'] = test_data['Parameter6']%test_data['Parameter8']
train_data['10%8'] = train_data['Parameter10']%train_data['Parameter8']
test_data['10%8'] = test_data['Parameter10']%test_data['Parameter8']

train_data['7*8'] = train_data['Parameter7'] * train_data['Parameter8']
test_data['7*8'] = test_data['Parameter7'] * test_data['Parameter8']
train_data['5*5'] = train_data['Parameter5'] * train_data['Parameter5']
test_data['5*5'] = test_data['Parameter5'] * test_data['Parameter5']
# train_data['6*6'] = train_data['Parameter6'] * train_data['Parameter6']
# test_data['6*6'] = test_data['Parameter6'] * test_data['Parameter6']
train_data['7*7'] = train_data['Parameter7'] * train_data['Parameter7']
test_data['7*7'] = test_data['Parameter7'] * test_data['Parameter7']
train_data['8*8'] = train_data['Parameter8'] * train_data['Parameter8']
test_data['8*8'] = test_data['Parameter8'] * test_data['Parameter8']
train_data['9*9'] = train_data['Parameter9'] * train_data['Parameter9']
test_data['9*9'] = test_data['Parameter9'] * test_data['Parameter9']
train_data['10*10'] = train_data['Parameter10'] * train_data['Parameter10']
test_data['10*10'] = test_data['Parameter10'] * test_data['Parameter10']

scale_features = ['7_9','8_9', '7_8', '9_10', '7_10', '9%8','7/8','7%8','8/9'
				  ]


# 组合所有特征
features = features + scale_features

print('all train features:', features)

for fc in features:
	n = train_data['{}'.format(fc)].nunique()
# 	这个的作用就是显示特征中的唯一值
	print(fc + ':', n)

all train features: ['Parameter5', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10', '7_9', '8_9', '7_8', '9_10', '7_10', '9%8', '7/8', '7%8', '8/9']
Parameter5: 132
Parameter7: 14
Parameter8: 23
Parameter9: 16
Parameter10: 41
7_9: 49
8_9: 52
7_8: 43
9_10: 260
7_10: 291
9%8: 49
7/8: 43
7%8: 41
8/9: 52


In [36]:
# train_data.shape

In [37]:
# train_data.sort_values('Parameter2', ascending=False)[:3]

In [38]:
# --------------------------------------------------------
# train
# --------------------------------------------------------
x_train = train_data[features].values
x_test = test_data[features].values

def lightgbm_model():
	model = LGBMClassifier(max_depth=5, learning_rate=0.01, n_estimators=1000, num_leaves=16,
						   objective='multiclass', silent=True,
						   reg_lambda=0.0001, #reg_alpha=1.,
						   #bagging_fraction=0.9, feature_fraction=0.9,
						   )
	return model

def xgboost_model():
	model = XGBClassifier(max_depth=5, n_estimators=1000, learning_rate=0.01, silent=True,min_child_weight=4,colsample_bytree=0.5,
						  #objective='multi:softmax'
						  )
	return model

def catboost_model():
	cbt_model = cbt.CatBoostClassifier(#iterations=100000, learning_rate=0.01, verbose=0,
		iterations=3000, learning_rate=0.01, verbose=0,
		max_depth=7, #reg_lambda=5.,
		task_type='GPU',   # cat_features=cat_list,
		loss_function='MultiClass',
# 		l2_leaf_reg = 7
		)
	return cbt_model

def rf_model():
	model = RandomForestClassifier(n_estimators=2000,max_depth=5,random_state=829,oob_score=True,min_samples_leaf=6,min_samples_split=3
						  )
	return model

def catboost_importance():
	model = catboost_model()
	model.fit(x_train, np.argmax(labels, 1))
	importance = model.get_feature_importance(prettified=True)
# 	这个其实就是类似PCA一样，找到特征的贡献
	print(importance.columns)
	for i in range(len(features)):
		print(features[int(importance['Feature Id'][i])] + ' :', importance['Importances'][i])

In [55]:
sclf = StackingClassifier(classifiers=[lightgbm_model(),xgboost_model(),catboost_model(),rf_model()],use_probas = True,meta_classifier = lr,average_probas = False)

In [56]:
def grid_search(mode):
	def my_custom_loss_func(y_true, y_pred):
		diff = np.mean(-y_true * np.log(y_pred + 1e-5))
		return diff
	score = make_scorer(my_custom_loss_func, greater_is_better=False)
	
	# grid search for lightgbm
	if mode=='lgb':
		parameters = {'max_depth': [5, 7, 9, 11], 'num_leaves': [16, 32, 64], 'reg_lambda': [7, 5,4, 3, 1, 0.1]}
		model = lightgbm_model()
	# grid search for catboost
	if mode=='cat':
		parameters = {'iterations': [3000, 4000, 5000],'max_depth': [ 6, 7, 8, 9],'learning_rate':[0.01, 0.008,0.005]}
		model = catboost_model()
	# grid search for xgboost
	if mode=='rf':
		parameters = {'n_estimators': [100, 500, 1000, 2000,3000],'max_depth': [5, 6, 7],'min_samples_leaf':[3,4,5,6],'min_samples_split':[3,4,5,6]}
		model = rf_model()
	if mode=='xgb':
		parameters = {'max_depth': [5, 7, 9, 11],'n_estimators':[1000, 2000, 3000], 
		'min_child_weight' : [1,2,3,4],'colsample_bytree': [0.5,0.8]}
		model = xgboost_model()
	if mode=='sclf':
	    parameters = {
	       # 'lgbmclassifier__max_depth':[5,6, 7, 9],'lgbmclassifier__num_leaves':[16, 32], 'lgbmclassifier__reg_lambda':[6,5,4,3],
	       #这个网格搜索必须要用小写，但是还是报错了
	       # 'catboostclassifier__max_depth':[6, 7, 8, 9],'catboostclassifier__learning_rate':[0.01,0.008],
	       # 'xgbclassifier__max_depth':[5,6,7],'xgbclassifier__min_child_weight':[3,4],'xgbclassifier__n_estimators':[1000,2000]
	       # 'classifiers':[(catboost_model(),catboost_model()),(catboost_model(),lightgbm_model(), catboost_model()), (catboost_model(), xgboost_model(), catboost_model())],
	    }
	    model = sclf
	# search
	clf = GridSearchCV(model, parameters, cv=5, scoring=score, verbose=2)
	clf.fit(x_train, np.argmax(labels, 1))
	print('best parameters:', clf.best_params_)
	print('best score:', clf.best_score_)
	

In [41]:
catboost_importance()

Index(['Feature Id', 'Importances'], dtype='object')
Parameter10 : 24.267509971537823
Parameter5 : 20.090112165080978
7_10 : 12.460654797417952
9_10 : 11.083609060881283
7/8 : 6.12377978137922
8/9 : 4.5603158575458185
9%8 : 4.184016809120137
7_9 : 3.9224417464041865
7%8 : 2.9325470864020664
8_9 : 2.876150105187855
7_8 : 2.7068207490551535
Parameter8 : 2.0746482020940413
Parameter9 : 1.723738235353265
Parameter7 : 0.9936554325402446


In [57]:
# k-fold train
def kfold_train(mode):
	acc_list, loss_list = [], []
	prediction = np.zeros((x_test.shape[0], 4))
	for i in range(10):
		print(str(i+1) + ' th kflod' + '*'*50)
		kf = KFold(n_splits=5, shuffle=True, random_state=i)
		kfold_list = []
		for k, (train_index, test_index) in enumerate(kf.split(x_train)):
			print(str(k+1) + 'fold--------------')
			train_x, train_y = x_train[train_index], labels[train_index]
			test_x, test_y = x_train[test_index], labels[test_index]
			# train
			if mode == 'cat':
				model = catboost_model()
				model.fit(train_x, np.argmax(train_y, 1), eval_set=(test_x, np.argmax(test_y, 1)),
					  #early_stopping_rounds=1000, verbose=False
						)
				#print(pd.DataFrame({'column': features, 'importance': model.feature_importances_}).sort_values(by='importance'))
			if mode == 'lgb':
				model = lightgbm_model()
				model.fit(train_x, np.argmax(train_y, 1), eval_set=(test_x, np.argmax(test_y, 1)),
					  # early_stopping_rounds=50, verbose=True
						  verbose=False
						  )
			if mode == 'xgb':
				model = xgboost_model()
				model.fit(train_x, np.argmax(train_y, 1), verbose=True, eval_set=(test_x, np.argmax(test_y, 1)))
			if mode == 'sclf':
				model = sclf
				model.fit(train_x, np.argmax(train_y, 1))
			if mode == 'rf':
				model = rf_model()
				model.fit(train_x, np.argmax(train_y, 1))
			pred = model.predict_proba(test_x)
			acc = accuracy_score(np.argmax(test_y, 1), np.argmax(pred, 1))
			loss = log_loss(test_y, pred)
			acc_list.append(acc)
			loss_list.append(loss)
			kfold_list.append(loss)
			print('test acc: %f, test loss: %f' % (acc, loss))
			# predict
			prediction += model.predict_proba(x_test)
# 			返回的是概率
		print('this fold mean loss:', np.mean(kfold_list))
	print('*'*50)
	print('mean acc: %f, mean loss: %f' % (np.mean(acc_list), np.mean(loss_list)))
	prediction = prediction / 50.
	return prediction

In [53]:
# from decimal import Decimal
def submit_result(prediction):
	sub = test_data[['Group']]
	prob_cols = [i for i in submit.columns if i not in ['Group']]
	for i, f in enumerate(prob_cols):
		sub[f] = prediction[:, i]
	for i in prob_cols:
		sub[i] = sub.groupby('Group')[i].transform('mean')
	sub = sub.drop_duplicates()
# 	for i in prob_cols:
	   # sub[i] = sub[i].apply(lambda x:(Decimal(str(x))//Decimal('0.02')+Decimal('1'))*Decimal('0.02') if Decimal(str(x))%Decimal('0.02') > Decimal('0.01') else x)
	   # sub[i] = sub[i].apply(lambda x:(Decimal(str(x))//Decimal('0.02'))*Decimal('0.02') if Decimal(str(x))%Decimal('0.02') < Decimal('0.01') else x)
	sub.to_csv("/home/aistudio/work/result.csv", index=False)

time1 = time.clock()
prediction = kfold_train('sclf')
time2 = time.clock()
print('running time: ', str((time2 - time1)/60))
submit_result(prediction)

1 th kflod**************************************************
1fold--------------


In [44]:
# grid_search('rf')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 